# Stock Price Prediction (LSTM)

Change `TICKER` below and run cells top-to-bottom.

In [ ]:

# --- Imports
import os, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Plot inline
%matplotlib inline

print("Libraries imported.")


In [ ]:

# --- Config
TICKER = "AAPL"
START_DATE = "2015-01-01"
END_DATE = "2023-12-31"
LOOKBACK = 60  # days

save_dir = "images"
os.makedirs(save_dir, exist_ok=True)


In [ ]:

# --- Download Data
df = yf.download(TICKER, start=START_DATE, end=END_DATE)
assert not df.empty, "No data downloaded. Check ticker or dates."
df.head()


In [ ]:

# --- Feature Engineering
data = df.copy()
data["MA20"] = data["Close"].rolling(window=20).mean()
data["MA50"] = data["Close"].rolling(window=50).mean()
data = data.dropna()
data.tail()


In [ ]:

# --- Scaling
features = data[["Close", "MA20", "MA50"]].values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(features)

def make_sequences(arr, lookback=60):
    X, y = [], []
    for i in range(lookback, len(arr)):
        X.append(arr[i - lookback:i])
        y.append(arr[i, 0])  # 'Close' is at index 0
    return np.array(X), np.array(y)

X, y = make_sequences(scaled, LOOKBACK)
print("Shapes -> X:", X.shape, "y:", y.shape)


In [ ]:

# --- Train / Test Split (80/20)
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]
len(X_train), len(X_test)


In [ ]:

# --- Build Model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.2),
    LSTM(64, return_sequences=False),
    Dropout(0.2),
    Dense(32, activation="relu"),
    Dense(1)
])

model.compile(optimizer="adam", loss="mse")
model.summary()


In [ ]:

# --- Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=32,
    verbose=1
)


In [ ]:

# --- Evaluate & Visualize
pred = model.predict(X_test).flatten()

plt.figure(figsize=(12, 6))
plt.plot(y_test, label="Actual")
plt.plot(pred, label="Predicted")
plt.title(f"{TICKER} — Actual vs Predicted (Test Set)")
plt.xlabel("Days")
plt.ylabel("Scaled Close Price")
plt.legend()
plot_path = os.path.join(save_dir, "result_plot.png")
plt.savefig(plot_path, bbox_inches="tight")
plot_path


In [ ]:

# --- Save Model
model_path = "model.h5"
model.save(model_path)
model_path
